In [1]:
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, LeakyReLU,Dropout,BatchNormalization,Dense,MaxPooling2D, Input, Flatten,Concatenate
from tensorflow.keras.regularizers import l2
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping

In [2]:
data='df_modif_auto.pkl'

with open(data, 'rb') as f:
    df_modif = pickle.load(f)

In [3]:
df_modif.head(3)

,image_id,label,path,outputs_256_to_128_norm,erreur_256_to_128_norm,outputs_256_to_128_pneumo,erreur_256_to_128_pneumo,outputs_256_to_128_covid,erreur_256_to_128_covid,outputs_128_to_256_norm,erreur_128_to_256_norm,outputs_128_to_256_pneumo,erreur_128_to_256_pneumo,outputs_128_to_256_covid,erreur_128_to_256_covid
0,7005867.png,0,data/data/7005867.png,"[[0.019909676, 0.020805862, 0.034866877, 0.052...","[[0.019909676, 0.020805862, 0.011337465, -0.06...","[[0.021528233, 0.023085557, 0.038397443, 0.057...","[[0.021528233, 0.023085557, 0.01486803, -0.056...","[[0.03838063, 0.04409813, 0.057933092, 0.07251...","[[0.03838063, 0.04409813, 0.03440368, -0.04121...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, -0.023529412, -0.0235294...","[[0.026406609, 0.028926425, 0.044291265, 0.044...","[[0.026406609, 0.028926425, 0.044291265, 0.044...","[[0.05270214, 0.05708629, 0.05802837, 0.078446...","[[0.05270214, 0.05708629, 0.05802837, 0.078446..."
1,6847606.png,0,data/data/6847606.png,"[[0.0188342, 0.015733546, 0.025393154, 0.04027...","[[0.0188342, 0.015733546, 0.025393154, 0.01674...","[[0.019359432, 0.01708153, 0.027140666, 0.0432...","[[0.019359432, 0.01708153, 0.027140666, 0.0197...","[[0.03213664, 0.034240626, 0.04511521, 0.05878...","[[0.03213664, 0.034240626, 0.04511521, 0.03525...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.003921569, ...","[[0.007432907, 0.004641678, 0.0017120203, 0.00...","[[0.007432907, 0.004641678, 0.0017120203, 0.00...","[[0.020323094, 0.020871565, 0.025175508, 0.034...","[[0.020323094, 0.020871565, 0.025175508, 0.034..."
2,4656855.png,1,data/data/4656855.png,"[[0.047677375, 0.08174259, 0.1257889, 0.158103...","[[-0.49222872, -0.47694284, -0.40003267, -0.29...","[[0.07091708, 0.12270764, 0.18051544, 0.221854...","[[-0.468989, -0.4359778, -0.34530613, -0.23354...","[[0.10213965, 0.13281164, 0.16514815, 0.191359...","[[-0.43776643, -0.4258738, -0.36067343, -0.264...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.53488374, -0.53953487, -0.5534884, -0.520...","[[0.21051943, 0.26774514, 0.29923058, 0.348166...","[[-0.3243643, -0.27178973, -0.2542578, -0.1727...","[[0.23558578, 0.26591277, 0.24990481, 0.334158...","[[-0.29929796, -0.2736221, -0.30358356, -0.186..."


In [4]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 761)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [12]:
class CustomDataGenerator_256_norm(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_norm_256 = self.df["outputs_128_to_256_norm"][i]
            #y_erreur_norm_256 = self.df["erreur_128_to_256_norm"][i]
            #pn_256 = np.resize(1000-(-y_pred_norm_256+y_erreur_norm_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_pred_norm_256),(*self.input_size,1))
            pn_256 = self.augment_input(pn_256)
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32')  
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y
    
    def augment_input(self,pn_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.12,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000) # generate a random seed
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
       
        return  pn_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_256_valid_norm(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_norm_256 = self.df["outputs_128_to_256_norm"][i]
            #y_erreur_norm_256 = self.df["erreur_128_to_256_norm"][i]
            #pn_256 = np.resize(1000-(-y_pred_norm_256+y_erreur_norm_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_pred_norm_256),(*self.input_size,1))                      
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32') 
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y

In [13]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_256_norm(train_all_df, batch_size=32)
valid_all_generator=CustomDataGenerator_256_valid_norm(val_all_df, batch_size=32)

In [16]:
def create_model(layer_names):
    inputs = Input(shape=(256, 256, 1), name=layer_names[0])

    x = Conv2D(filters=32, kernel_size=2, padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[1])(inputs)
    x = BatchNormalization(name=layer_names[2])(x)
    x = LeakyReLU(name=layer_names[3])(x)
    x = MaxPooling2D(2, 2, name=layer_names[4])(x)

    x = Conv2D(filters=64, kernel_size=3, padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[5])(x)
    x = BatchNormalization(name=layer_names[6])(x)
    x = LeakyReLU(name=layer_names[7])(x)
    x = MaxPooling2D(3, 3, name=layer_names[8])(x)

    x = Conv2D(filters=256, kernel_size=3, padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[9])(x)
    x = BatchNormalization(name=layer_names[10])(x)
    x = LeakyReLU(name=layer_names[11])(x)
    x = MaxPooling2D(2, 2, name=layer_names[12])(x)

    x = Flatten(name=layer_names[13])(x)
    x = Dense(256, kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[14])(x)
    x = LeakyReLU(name=layer_names[15])(x)
    x = Dropout(0.2, name=layer_names[16])(x)

    x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[17])(x)
    x = LeakyReLU(name=layer_names[18])(x)
    x = Dropout(0.25, name=layer_names[19])(x)

    x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[20])(x)
    x = LeakyReLU(name=layer_names[21])(x)
    x = Dropout(0.3, name=layer_names[22])(x)

    x = Dense(32, kernel_initializer='he_normal', kernel_regularizer=l2(10), name=layer_names[23])(x)
    x = LeakyReLU(name=layer_names[24])(x)
    x = BatchNormalization(name=layer_names[25])(x)
    x = Dropout(0.3, name=layer_names[26])(x)

    outputs = Dense(3, activation='softmax', name=layer_names[27])(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [17]:
layer_names_norm = ['input_norm'] + ['norm_{}'.format(i) for i in range(1, 27)] + ['output_norm']

ChestX_norm= create_model(layer_names_norm)

# Create Adamax optimizer with learning rate 0.00001 and momentum 0.9
optimizer = Adamax(learning_rate=0.00001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=15, verbose=1, restore_best_weights=True)
ChestX_norm.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_norm.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_norm (InputLayer)     [(None, 256, 256, 1)]     0         
                                                                 
 norm_1 (Conv2D)             (None, 255, 255, 32)      160       
                                                                 
 norm_2 (BatchNormalization)  (None, 255, 255, 32)     128       
                                                                 
 norm_3 (LeakyReLU)          (None, 255, 255, 32)      0         
                                                                 
 norm_4 (MaxPooling2D)       (None, 127, 127, 32)      0         
                                                                 
 norm_5 (Conv2D)             (None, 125, 125, 64)      18496     
                                                                 
 norm_6 (BatchNormalization)  (None, 125, 125, 64)     256 

In [18]:
history_ChestX_norm = ChestX_norm.fit(train_all_generator, epochs=100, validation_data=valid_all_generator, callbacks= early_stop)

Epoch 1/100
74/74 [==============================] - 9s 107ms/step - loss: 16131.4365 - accuracy: 0.4135 - val_loss: 15562.0957 - val_accuracy: 0.5939
Epoch 2/100
74/74 [==============================] - 8s 104ms/step - loss: 15076.8496 - accuracy: 0.5100 - val_loss: 14605.3613 - val_accuracy: 0.5324
Epoch 3/100
74/74 [==============================] - 8s 104ms/step - loss: 14196.5723 - accuracy: 0.5378 - val_loss: 13798.7637 - val_accuracy: 0.5324
Epoch 4/100
74/74 [==============================] - 8s 104ms/step - loss: 13447.3535 - accuracy: 0.5369 - val_loss: 13098.4248 - val_accuracy: 0.5631
Epoch 5/100
74/74 [==============================] - 8s 104ms/step - loss: 12790.6016 - accuracy: 0.5267 - val_loss: 12491.2646 - val_accuracy: 0.5631
Epoch 6/100
74/74 [==============================] - 8s 104ms/step - loss: 12226.7744 - accuracy: 0.4947 - val_loss: 11961.4004 - val_accuracy: 0.6246
Epoch 7/100
74/74 [==============================] - 8s 104ms/step - loss: 11730.2139 - accura

In [20]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    y_pred_norm_256 = df_all_test["outputs_128_to_256_norm"][i]
    #y_erreur_norm_256 = df_all_test["erreur_128_to_256_norm"][i]
    #pn_256 = np.resize(1000-(-y_pred_norm_256+y_erreur_norm_256), (*(256,256),1))
    pn_256 = np.resize(1000-(y_pred_norm_256),(*(256,256),1))
    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_y = y

    y_pred = ChestX_norm.predict(batch_pn_256,verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)

In [21]:
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      224   34    2
1       14  227    6
2       13   19  193


In [22]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 158)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [23]:
class CustomDataGenerator_256_pneumo(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_pneumo_256 = self.df["outputs_128_to_256_pneumo"][i]
            #y_erreur_pneumo_256 = self.df["erreur_128_to_256_pneumo"][i]
            #pn_256 = np.resize(1000-(-y_pred_pneumo_256+y_erreur_pneumo_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_pred_pneumo_256),(*self.input_size,1))
            pn_256 = self.augment_input(pn_256)
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32')  
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y
    
    def augment_input(self,pn_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.12,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000) # generate a random seed
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
       
        return  pn_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_256_valid_pneumo(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_pneumo_256 = self.df["outputs_128_to_256_pneumo"][i]
            #y_erreur_pneumo_256 = self.df["erreur_128_to_256_pneumo"][i]
            #pn_256 = np.resize(1000-(-y_pred_pneumo_256+y_erreur_pneumo_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_pred_pneumo_256),(*self.input_size,1))                      
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32') 
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y

In [24]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_256_pneumo(train_all_df, batch_size=32)
valid_all_generator=CustomDataGenerator_256_valid_pneumo(val_all_df, batch_size=32)

In [25]:
layer_names_pneumo = ['input_pneumo'] + ['pneumo_{}'.format(i) for i in range(1, 27)] + ['output_pneumo']

ChestX_pneumo= create_model(layer_names_pneumo)
optimizer = Adamax(learning_rate=0.00001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)
ChestX_pneumo.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_pneumo.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_pneumo (InputLayer)   [(None, 256, 256, 1)]     0         
                                                                 
 pneumo_1 (Conv2D)           (None, 255, 255, 32)      160       
                                                                 
 pneumo_2 (BatchNormalizatio  (None, 255, 255, 32)     128       
 n)                                                              
                                                                 
 pneumo_3 (LeakyReLU)        (None, 255, 255, 32)      0         
                                                                 
 pneumo_4 (MaxPooling2D)     (None, 127, 127, 32)      0         
                                                                 
 pneumo_5 (Conv2D)           (None, 125, 125, 64)      18496     
                                                           

In [26]:
history_ChestX_pneumo = ChestX_pneumo.fit(train_all_generator, epochs=100, validation_data=valid_all_generator, callbacks= early_stop)

Epoch 1/100
74/74 [==============================] - 9s 109ms/step - loss: 15994.7881 - accuracy: 0.5023 - val_loss: 15451.4365 - val_accuracy: 0.4727
Epoch 2/100
74/74 [==============================] - 8s 107ms/step - loss: 14990.8076 - accuracy: 0.6138 - val_loss: 14529.8721 - val_accuracy: 0.6860
Epoch 3/100
74/74 [==============================] - 8s 107ms/step - loss: 14120.9941 - accuracy: 0.5720 - val_loss: 13722.8154 - val_accuracy: 0.6877
Epoch 4/100
74/74 [==============================] - 8s 105ms/step - loss: 13368.2314 - accuracy: 0.5681 - val_loss: 13015.3750 - val_accuracy: 0.7491
Epoch 5/100
74/74 [==============================] - 8s 104ms/step - loss: 12702.3164 - accuracy: 0.5451 - val_loss: 12397.3311 - val_accuracy: 0.7201
Epoch 6/100
74/74 [==============================] - 8s 105ms/step - loss: 12123.4082 - accuracy: 0.5139 - val_loss: 11856.8857 - val_accuracy: 0.8123
Epoch 7/100
74/74 [==============================] - 8s 104ms/step - loss: 11618.3291 - accura

In [27]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    y_pred_pneumo_256 = df_all_test["outputs_128_to_256_pneumo"][i]
    #y_erreur_pneumo_256 = df_all_test["erreur_128_to_256_pneumo"][i]
    #pn_256 = np.resize(1000-(-y_pred_pneumo_256+y_erreur_pneumo_256), (*(256,256),1))
    pn_256 = np.resize(1000-(y_pred_pneumo_256),(*(256,256),1))
    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_y = y

    y_pred = ChestX_pneumo.predict(batch_pn_256,verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)
    
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      211   41    8
1       24  209   14
2        6    8  211


In [28]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 370)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [29]:
class CustomDataGenerator_256_covid(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_covid_256 = self.df["outputs_128_to_256_covid"][i]
            #y_erreur_covid_256 = self.df["erreur_128_to_256_covid"][i]
            #pn_256 = np.resize(1000-(-y_pred_covid_256+y_erreur_covid_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_pred_covid_256),(*self.input_size,1))
            pn_256 = self.augment_input(pn_256)
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32')  
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y
    
    def augment_input(self,pn_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.12,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000) # generate a random seed
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
       
        return  pn_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_256_valid_covid(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_covid_256 = self.df["outputs_128_to_256_covid"][i]
            #y_erreur_covid_256 = self.df["erreur_128_to_256_covid"][i]
            #pn_256 = np.resize(1000-(-y_pred_covid_256+y_erreur_covid_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_pred_covid_256),(*self.input_size,1))                      
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32') 
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y

In [30]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_256_covid(train_all_df, batch_size=32)
valid_all_generator=CustomDataGenerator_256_valid_covid(val_all_df, batch_size=32)

In [31]:
layer_names_covid = ['input_covid'] + ['covid_{}'.format(i) for i in range(1, 27)] + ['output_covid']

ChestX_covid= create_model(layer_names_covid)
optimizer = Adamax(learning_rate=0.00001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)
ChestX_covid.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_covid.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_covid (InputLayer)    [(None, 256, 256, 1)]     0         
                                                                 
 covid_1 (Conv2D)            (None, 255, 255, 32)      160       
                                                                 
 covid_2 (BatchNormalization  (None, 255, 255, 32)     128       
 )                                                               
                                                                 
 covid_3 (LeakyReLU)         (None, 255, 255, 32)      0         
                                                                 
 covid_4 (MaxPooling2D)      (None, 127, 127, 32)      0         
                                                                 
 covid_5 (Conv2D)            (None, 125, 125, 64)      18496     
                                                           

In [32]:
history_ChestX_covid = ChestX_covid.fit(train_all_generator, epochs=100, validation_data=valid_all_generator, callbacks= early_stop)

Epoch 1/100
74/74 [==============================] - 9s 108ms/step - loss: 16133.7754 - accuracy: 0.5442 - val_loss: 15581.4727 - val_accuracy: 0.4061
Epoch 2/100
74/74 [==============================] - 8s 105ms/step - loss: 15098.1670 - accuracy: 0.5968 - val_loss: 14627.2803 - val_accuracy: 0.5939
Epoch 3/100
74/74 [==============================] - 8s 105ms/step - loss: 14217.8555 - accuracy: 0.6416 - val_loss: 13812.4238 - val_accuracy: 0.6263
Epoch 4/100
74/74 [==============================] - 8s 105ms/step - loss: 13457.0508 - accuracy: 0.6574 - val_loss: 13103.9521 - val_accuracy: 0.8123
Epoch 5/100
74/74 [==============================] - 8s 104ms/step - loss: 12791.1943 - accuracy: 0.6455 - val_loss: 12487.2480 - val_accuracy: 0.7833
Epoch 6/100
74/74 [==============================] - 8s 105ms/step - loss: 12214.9199 - accuracy: 0.6463 - val_loss: 11950.1279 - val_accuracy: 0.8140
Epoch 7/100
74/74 [==============================] - 8s 105ms/step - loss: 11713.1777 - accura

In [33]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    y_pred_covid_256 = df_all_test["outputs_128_to_256_covid"][i]
    #y_erreur_covid_256 = df_all_test["erreur_128_to_256_covid"][i]
    #pn_256 = np.resize(1000-(-y_pred_covid_256+y_erreur_covid_256), (*(256,256),1))
    pn_256 = np.resize(1000-(y_pred_covid_256),(*(256,256),1))
    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_y = y

    y_pred = ChestX_covid.predict(batch_pn_256,verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)
    
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      233   24    3
1       27  219    1
2       23   24  178


In [34]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 652)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [35]:
class CustomDataGenerator_256_err_covid(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            #y_pred_covid_256 = self.df["outputs_128_to_256_covid"][i]
            y_erreur_covid_256 = self.df["erreur_128_to_256_covid"][i]
            #pn_256 = np.resize(1000-(-y_pred_covid_256+y_erreur_covid_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_erreur_covid_256),(*self.input_size,1))
            pn_256 = self.augment_input(pn_256)
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32')  
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y
    
    def augment_input(self,pn_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.12,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000) # generate a random seed
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
       
        return  pn_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_256_valid_err_covid(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            #y_pred_covid_256 = self.df["outputs_128_to_256_covid"][i]
            y_erreur_covid_256 = self.df["erreur_128_to_256_covid"][i]
            #pn_256 = np.resize(1000-(-y_pred_covid_256+y_erreur_covid_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_erreur_covid_256),(*self.input_size,1))                      
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32') 
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y

In [36]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_256_err_covid(train_all_df, batch_size=32)
valid_all_generator=CustomDataGenerator_256_valid_err_covid(val_all_df, batch_size=32)

In [37]:
layer_names_err_covid = ['input_err_covid'] + ['err_covid_{}'.format(i) for i in range(1, 27)] + ['output_err_covid']

ChestX_err_covid= create_model(layer_names_err_covid)
optimizer = Adamax(learning_rate=0.00001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)
ChestX_err_covid.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_err_covid.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_err_covid (InputLayer  [(None, 256, 256, 1)]    0         
 )                                                               
                                                                 
 err_covid_1 (Conv2D)        (None, 255, 255, 32)      160       
                                                                 
 err_covid_2 (BatchNormaliza  (None, 255, 255, 32)     128       
 tion)                                                           
                                                                 
 err_covid_3 (LeakyReLU)     (None, 255, 255, 32)      0         
                                                                 
 err_covid_4 (MaxPooling2D)  (None, 127, 127, 32)      0         
                                                                 
 err_covid_5 (Conv2D)        (None, 125, 125, 64)      1849

In [38]:
history_ChestX_err_covid = ChestX_err_covid.fit(train_all_generator, epochs=100, validation_data=valid_all_generator, callbacks= early_stop)

Epoch 1/100
74/74 [==============================] - 9s 107ms/step - loss: 16111.9111 - accuracy: 0.4361 - val_loss: 15573.0430 - val_accuracy: 0.2850
Epoch 2/100
74/74 [==============================] - 8s 104ms/step - loss: 15102.3037 - accuracy: 0.4737 - val_loss: 14640.5869 - val_accuracy: 0.2850
Epoch 3/100
74/74 [==============================] - 8s 106ms/step - loss: 14231.9180 - accuracy: 0.4925 - val_loss: 13827.6230 - val_accuracy: 0.2850
Epoch 4/100
74/74 [==============================] - 9s 115ms/step - loss: 13472.4268 - accuracy: 0.4823 - val_loss: 13116.6230 - val_accuracy: 0.2833
Epoch 5/100
74/74 [==============================] - 8s 103ms/step - loss: 12802.8037 - accuracy: 0.4900 - val_loss: 12494.7734 - val_accuracy: 0.3140
Epoch 6/100
74/74 [==============================] - 8s 110ms/step - loss: 12222.1758 - accuracy: 0.4507 - val_loss: 11951.9512 - val_accuracy: 0.3430
Epoch 7/100
74/74 [==============================] - 8s 112ms/step - loss: 11712.2832 - accura

In [39]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    #y_pred_covid_256 = df_all_test["outputs_128_to_256_covid"][i]
    y_erreur_covid_256 = df_all_test["erreur_128_to_256_covid"][i]
    #pn_256 = np.resize(1000-(-y_pred_covid_256+y_erreur_covid_256), (*(256,256),1))
    pn_256 = np.resize(1000-(y_erreur_covid_256),(*(256,256),1))
    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_y = y

    y_pred = ChestX_err_covid.predict(batch_pn_256,verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)
    
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      136  121    3
1      112  130    5
2       65   10  150


In [40]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 798)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [41]:
class CustomDataGenerator_256_err_norm(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            #y_pred_norm_256 = self.df["outputs_128_to_256_norm"][i]
            y_erreur_norm_256 = self.df["erreur_128_to_256_norm"][i]
            #pn_256 = np.resize(1000-(-y_pred_norm_256+y_erreur_norm_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_erreur_norm_256),(*self.input_size,1))
            pn_256 = self.augment_input(pn_256)
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32')  
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y
    
    def augment_input(self,pn_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.12,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000) # generate a random seed
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
       
        return  pn_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_256_valid_err_norm(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            #y_pred_norm_256 = self.df["outputs_128_to_256_norm"][i]
            y_erreur_norm_256 = self.df["erreur_128_to_256_norm"][i]
            #pn_256 = np.resize(1000-(-y_pred_norm_256+y_erreur_norm_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_erreur_norm_256),(*self.input_size,1))                      
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32') 
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y

In [42]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_256_err_norm(train_all_df, batch_size=32)
valid_all_generator=CustomDataGenerator_256_valid_err_norm(val_all_df, batch_size=32)

In [43]:
layer_names_err_norm = ['input_err_norm'] + ['err_norm_{}'.format(i) for i in range(1, 27)] + ['output_err_norm']

ChestX_err_norm= create_model(layer_names_err_norm)
optimizer = Adamax(learning_rate=0.00001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)
ChestX_err_norm.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_err_norm.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_err_norm (InputLayer)  [(None, 256, 256, 1)]    0         
                                                                 
 err_norm_1 (Conv2D)         (None, 255, 255, 32)      160       
                                                                 
 err_norm_2 (BatchNormalizat  (None, 255, 255, 32)     128       
 ion)                                                            
                                                                 
 err_norm_3 (LeakyReLU)      (None, 255, 255, 32)      0         
                                                                 
 err_norm_4 (MaxPooling2D)   (None, 127, 127, 32)      0         
                                                                 
 err_norm_5 (Conv2D)         (None, 125, 125, 64)      18496     
                                                           

In [44]:
history_ChestX_err_norm = ChestX_err_norm.fit(train_all_generator, epochs=100, validation_data=valid_all_generator, callbacks= early_stop)

Epoch 1/100
74/74 [==============================] - 9s 107ms/step - loss: 15971.3252 - accuracy: 0.4229 - val_loss: 15391.5615 - val_accuracy: 0.3447
Epoch 2/100
74/74 [==============================] - 8s 104ms/step - loss: 14901.5752 - accuracy: 0.5267 - val_loss: 14420.1162 - val_accuracy: 0.3447
Epoch 3/100
74/74 [==============================] - 8s 104ms/step - loss: 14001.9756 - accuracy: 0.5886 - val_loss: 13600.5791 - val_accuracy: 0.3447
Epoch 4/100
74/74 [==============================] - 8s 106ms/step - loss: 13246.2168 - accuracy: 0.6091 - val_loss: 12895.9521 - val_accuracy: 0.3754
Epoch 5/100
74/74 [==============================] - 8s 105ms/step - loss: 12593.3154 - accuracy: 0.6446 - val_loss: 12294.3867 - val_accuracy: 0.3754
Epoch 6/100
74/74 [==============================] - 8s 104ms/step - loss: 12028.0928 - accuracy: 0.6557 - val_loss: 11772.7764 - val_accuracy: 0.5648
Epoch 7/100
74/74 [==============================] - 8s 105ms/step - loss: 11544.3887 - accura

In [45]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    #y_pred_norm_256 = df_all_test["outputs_128_to_256_norm"][i]
    y_erreur_norm_256 = df_all_test["erreur_128_to_256_norm"][i]
    #pn_256 = np.resize(1000-(-y_pred_norm_256+y_erreur_norm_256), (*(256,256),1))
    pn_256 = np.resize(1000-(y_erreur_norm_256),(*(256,256),1))
    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_y = y

    y_pred = ChestX_err_norm.predict(batch_pn_256,verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)
    
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      229   30    1
1       65  152   30
2       42    7  176


In [46]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 895)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [47]:
class CustomDataGenerator_256_err_pneumo(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            #y_pred_pneumo_256 = self.df["outputs_128_to_256_pneumo"][i]
            y_erreur_pneumo_256 = self.df["erreur_128_to_256_pneumo"][i]
            #pn_256 = np.resize(1000-(-y_pred_pneumo_256+y_erreur_pneumo_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_erreur_pneumo_256),(*self.input_size,1))
            pn_256 = self.augment_input(pn_256)
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32')  
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y
    
    def augment_input(self,pn_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.12,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000) # generate a random seed
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
       
        return  pn_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_256_valid_err_pneumo(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_y = np.zeros((len(batch_df), self.num_classes))

        for i, row in enumerate(batch_df.itertuples()):
            
            #y_pred_pneumo_256 = self.df["outputs_128_to_256_pneumo"][i]
            y_erreur_pneumo_256 = self.df["erreur_128_to_256_pneumo"][i]
            #pn_256 = np.resize(1000-(-y_pred_pneumo_256+y_erreur_pneumo_256), (*self.input_size,1))
            pn_256 = np.resize(1000-(y_erreur_pneumo_256),(*self.input_size,1))                      
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*(self.input_size),1)).astype('float32') 
            
            #rgb_img = np.zeros((pn_256.shape[0], pn_256.shape[1], 3), dtype=np.uint8)

            # Replicate the grayscale channel to all three channels of the RGB image
            #rgb_img[:,:,0] = pn_256
            #rgb_img[:,:,1] = pn_256
            #rgb_img[:,:,2] = pn_256

            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0
         

            batch_pn_256[i] = pn_256
            batch_y[i] = y
        
        return batch_pn_256, batch_y

In [48]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_256_err_pneumo(train_all_df, batch_size=32)
valid_all_generator=CustomDataGenerator_256_valid_err_pneumo(val_all_df, batch_size=32)

In [49]:
layer_names_err_pneumo = ['input_err_pneumo'] + ['err_pneumo_{}'.format(i) for i in range(1, 27)] + ['output_err_pneumo']

ChestX_err_pneumo= create_model(layer_names_err_pneumo)
optimizer = Adamax(learning_rate=0.00001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)
ChestX_err_pneumo.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_err_pneumo.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_err_pneumo (InputLaye  [(None, 256, 256, 1)]    0         
 r)                                                              
                                                                 
 err_pneumo_1 (Conv2D)       (None, 255, 255, 32)      160       
                                                                 
 err_pneumo_2 (BatchNormaliz  (None, 255, 255, 32)     128       
 ation)                                                          
                                                                 
 err_pneumo_3 (LeakyReLU)    (None, 255, 255, 32)      0         
                                                                 
 err_pneumo_4 (MaxPooling2D)  (None, 127, 127, 32)     0         
                                                                 
 err_pneumo_5 (Conv2D)       (None, 125, 125, 64)      1849

In [50]:
history_ChestX_err_pneumo = ChestX_err_pneumo.fit(train_all_generator, epochs=100, validation_data=valid_all_generator, callbacks= early_stop)

Epoch 1/100
74/74 [==============================] - 9s 109ms/step - loss: 15956.6719 - accuracy: 0.4079 - val_loss: 15387.3926 - val_accuracy: 0.4710
Epoch 2/100
74/74 [==============================] - 8s 105ms/step - loss: 14913.9873 - accuracy: 0.4836 - val_loss: 14451.2197 - val_accuracy: 0.3413
Epoch 3/100
74/74 [==============================] - 8s 105ms/step - loss: 14038.3203 - accuracy: 0.4938 - val_loss: 13641.1055 - val_accuracy: 0.5939
Epoch 4/100
74/74 [==============================] - 8s 104ms/step - loss: 13283.9189 - accuracy: 0.4387 - val_loss: 12935.8281 - val_accuracy: 0.5614
Epoch 5/100
74/74 [==============================] - 8s 104ms/step - loss: 12627.6250 - accuracy: 0.4720 - val_loss: 12320.8750 - val_accuracy: 0.5922
Epoch 6/100
74/74 [==============================] - 8s 105ms/step - loss: 12050.4785 - accuracy: 0.4443 - val_loss: 11785.6104 - val_accuracy: 0.6894
Epoch 7/100
74/74 [==============================] - 8s 104ms/step - loss: 11549.6143 - accura

In [51]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    #y_pred_pneumo_256 = df_all_test["outputs_128_to_256_pneumo"][i]
    y_erreur_pneumo_256 = df_all_test["erreur_128_to_256_pneumo"][i]
    #pn_256 = np.resize(1000-(-y_pred_pneumo_256+y_erreur_pneumo_256), (*(256,256),1))
    pn_256 = np.resize(1000-(y_erreur_pneumo_256),(*(256,256),1))
    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_y = y

    y_pred = ChestX_err_pneumo.predict(batch_pn_256,verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)
    
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      198   50   12
1       41  197    9
2        7   41  177


In [142]:
df_all_train, df_all_test = train_test_split(df_modif, test_size=0.2, random_state=168)
train_all_df, val_all_df = train_test_split(df_all_train, test_size=0.2, random_state= 761)
# Afficher le nombre de lignes de chaque ensemble de données
print('Nombre de lignes dans l\'ensemble d\'entraînement et de validation :', len(df_all_train))
print('Nombre de lignes dans l\'ensemble de test :', len(df_all_test))
print('Nombre de lignes dans l\'ensemble d\'entraînement :', len(train_all_df))
print('Nombre de lignes dans l\'ensemble de validation :', len(val_all_df))

Nombre de lignes dans l'ensemble d'entraînement et de validation : 2927
Nombre de lignes dans l'ensemble de test : 732
Nombre de lignes dans l'ensemble d'entraînement : 2341
Nombre de lignes dans l'ensemble de validation : 586


In [143]:
class CustomDataGenerator_all(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        self.num_classes = 3 
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_y = np.zeros((len(batch_df), self.num_classes))
        
        
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_pp_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_pc_256 = np.zeros((len(batch_df), *self.input_size, 1))
        
        batch_en_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_ep_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_ec_256 = np.zeros((len(batch_df), *self.input_size, 1))
        
        batch_y = np.zeros((len(batch_df), self.num_classes))
        
        for i, row in enumerate(batch_df.itertuples()):
            
            
            y_pred_norm_256 = self.df["outputs_128_to_256_norm"][i]
            y_pred_pneumo_256 = self.df["outputs_128_to_256_pneumo"][i]
            y_pred_covid_256 = self.df["outputs_128_to_256_covid"][i]
            
            y_erreur_norm_256 = self.df["erreur_128_to_256_norm"][i]
            y_erreur_pneumo_256 = self.df["erreur_128_to_256_pneumo"][i]
            y_erreur_covid_256 = self.df["erreur_128_to_256_covid"][i]
            
            y_pred_norm_256 = np.resize(1000-(y_pred_norm_256),(*self.input_size,1))
            y_pred_pneumo_256 = np.resize(1000-(y_pred_pneumo_256),(*self.input_size,1))
            y_pred_covid_256 = np.resize(1000-(y_pred_covid_256),(*self.input_size,1))
            
            y_erreur_norm_256 = np.resize(1000-(y_erreur_norm_256),(*self.input_size,1))
            y_erreur_pneumo_256 = np.resize(1000-(y_erreur_pneumo_256),(*self.input_size,1))
            y_erreur_covid_256 = np.resize(1000-(y_erreur_covid_256),(*self.input_size,1))
                                   
            pn_256, pp_256, pc_256,en_256, ep_256,ec_256 = self.augment_input(y_pred_norm_256, y_pred_pneumo_256, y_pred_covid_256,
                                                                              y_erreur_norm_256, y_erreur_pneumo_256, y_erreur_covid_256)

            
            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*self.input_size,1)).astype('float32') 
            
            pp_256 = (pp_256- np.min(pp_256)) / (np.max(pp_256) - np.min(pp_256))
            pp_256 = np.reshape(pp_256,(*self.input_size,1)).astype('float32') 
            
            pc_256 = (pc_256- np.min(pc_256)) / (np.max(pc_256) - np.min(pc_256))
            pc_256 = np.reshape(pc_256,(*self.input_size,1)).astype('float32') 
            
            en_256 = (en_256- np.min(en_256)) / (np.max(en_256) - np.min(en_256))
            en_256 = np.reshape(en_256,(*self.input_size,1)).astype('float32') 
            
            ep_256 = (ep_256- np.min(ep_256)) / (np.max(ep_256) - np.min(ep_256))
            ep_256 = np.reshape(ep_256,(*self.input_size,1)).astype('float32') 
            
            ec_256 = (ec_256- np.min(ec_256)) / (np.max(ec_256) - np.min(ec_256))
            ec_256 = np.reshape(ec_256,(*self.input_size,1)).astype('float32') 
            
            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0

            batch_pn_256[i] = pn_256
            batch_pp_256[i] = pp_256
            batch_pc_256[i] = pc_256

            batch_en_256[i] = en_256
            batch_ep_256[i] = ep_256
            batch_ec_256[i] = ec_256
            
            batch_y[i] = y
        
        return [batch_pn_256,batch_pp_256,batch_pc_256,batch_en_256,batch_ep_256,batch_ec_256], batch_y
    
    def augment_input(self,pn_256, pp_256, pc_256,en_256, ep_256,ec_256):
    # Apply data augmentation to the input images x1 and x2
        image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=20,  # rotation jusqu'à 30° 
            zoom_range=0.15,     # zoom juqu'à 10%
            brightness_range=[0.85, 1.15], # luminosité impa
            horizontal_flip=True,
            fill_mode='nearest', # mode de completion de l'image modifié
        )
        seed = np.random.randint(0, 10000)
        pn_256 = image_generator.random_transform(pn_256, seed=seed)
        pp_256 = image_generator.random_transform(pp_256, seed=seed)
        pc_256 = image_generator.random_transform(pc_256, seed=seed)
        en_256 = image_generator.random_transform(en_256, seed=seed)
        ep_256 = image_generator.random_transform(ep_256, seed=seed)
        ec_256 = image_generator.random_transform(ec_256, seed=seed)
        return pn_256, pp_256, pc_256,en_256, ep_256,ec_256
    
    def on_epoch_end(self):
        # melange le dataframe à la fin de chaque epoch d'entrainement
        if self.shuffle:
            self.df = self.df.sample(frac=1)
            
class CustomDataGenerator_all_valid(Sequence):
    
    def __init__(self, df, batch_size, input_size=(256,256), shuffle=False):
        self.df = df
        self.batch_size = batch_size
        self.input_size = input_size
        self.num_classes = 3 
        self.shuffle = shuffle
    
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        
        batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
        batch_y = np.zeros((len(batch_df), self.num_classes))
        
        
        batch_pn_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_pp_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_pc_256 = np.zeros((len(batch_df), *self.input_size, 1))
        
        batch_en_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_ep_256 = np.zeros((len(batch_df), *self.input_size, 1))
        batch_ec_256 = np.zeros((len(batch_df), *self.input_size, 1))
        
        batch_y = np.zeros((len(batch_df), self.num_classes))
        
        for i, row in enumerate(batch_df.itertuples()):
            
            y_pred_norm_256 = self.df["outputs_128_to_256_norm"][i]
            y_pred_pneumo_256 = self.df["outputs_128_to_256_pneumo"][i]
            y_pred_covid_256 = self.df["outputs_128_to_256_covid"][i]
            
            y_erreur_norm_256 = self.df["erreur_128_to_256_norm"][i]
            y_erreur_pneumo_256 = self.df["erreur_128_to_256_pneumo"][i]
            y_erreur_covid_256 = self.df["erreur_128_to_256_covid"][i]
            
            pn_256 = np.resize(1000-(y_pred_norm_256),(*self.input_size,1))
            pp_256 = np.resize(1000-(y_pred_pneumo_256),(*self.input_size,1))
            pc_256 = np.resize(1000-(y_pred_covid_256),(*self.input_size,1))
            
            en_256 = np.resize(1000-(y_erreur_norm_256),(*self.input_size,1))
            ep_256 = np.resize(1000-(y_erreur_pneumo_256),(*self.input_size,1))
            ec_256 = np.resize(1000-(y_erreur_covid_256),(*self.input_size,1))
                                   

            pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
            pn_256 = np.reshape(pn_256,(*self.input_size,1)).astype('float32') 
            
            pp_256 = (pp_256- np.min(pp_256)) / (np.max(pp_256) - np.min(pp_256))
            pp_256 = np.reshape(pp_256,(*self.input_size,1)).astype('float32') 
            
            pc_256 = (pc_256- np.min(pc_256)) / (np.max(pc_256) - np.min(pc_256))
            pc_256 = np.reshape(pc_256,(*self.input_size,1)).astype('float32') 
            
            en_256 = (en_256- np.min(en_256)) / (np.max(en_256) - np.min(en_256))
            en_256 = np.reshape(en_256,(*self.input_size,1)).astype('float32') 
            
            ep_256 = (ep_256- np.min(ep_256)) / (np.max(ep_256) - np.min(ep_256))
            ep_256 = np.reshape(ep_256,(*self.input_size,1)).astype('float32') 
            
            ec_256 = (ec_256- np.min(ec_256)) / (np.max(ec_256) - np.min(ec_256))
            ec_256 = np.reshape(ec_256,(*self.input_size,1)).astype('float32') 
            
            
            # label encodé en one-hot
            label = self.df["label"][i]
            y = np.zeros(self.num_classes)
            y[label] = 1.0

            batch_pn_256[i] = pn_256
            batch_pp_256[i] = pp_256
            batch_pc_256[i] = pc_256

            batch_en_256[i] = en_256
            batch_ep_256[i] = ep_256
            batch_ec_256[i] = ec_256
            
            batch_y[i] = y
        
        return [batch_pn_256,batch_pp_256,batch_pc_256,batch_en_256,batch_ep_256,batch_ec_256], batch_y

In [144]:
train_all_df = train_all_df.reset_index(drop= True)
val_all_df = val_all_df.reset_index(drop= True)

train_all_generator=CustomDataGenerator_all(train_all_df, batch_size=4)
valid_all_generator=CustomDataGenerator_all_valid(val_all_df, batch_size=4)

In [145]:
predicted_input_norm_256 = Input(shape=(256, 256, 1))
predicted_input_pneumo_256 = Input(shape=(256, 256, 1))
predicted_input_covid_256 = Input(shape=(256, 256, 1))

error_input_norm_256 = Input(shape=(256, 256, 1))
error_input_pneumo_256 = Input(shape=(256, 256, 1))
error_input_covid_256 = Input(shape=(256, 256, 1))

net_err_norm = ChestX_err_norm(error_input_norm_256)
net_err_pneumo = ChestX_err_pneumo(error_input_pneumo_256)
net_err_covid = ChestX_err_covid(error_input_covid_256)

net_norm = ChestX_norm(predicted_input_norm_256)
net_pneumo = ChestX_pneumo(predicted_input_pneumo_256)
net_covid = ChestX_covid(predicted_input_covid_256)

for layer in ChestX_err_norm.layers:
    layer.trainable = False
    
for layer in ChestX_err_pneumo.layers:
    layer.trainable = False

for layer in ChestX_err_covid.layers:
    layer.trainable = False
    
for layer in ChestX_norm.layers:
    layer.trainable = False
    
for layer in ChestX_pneumo.layers:
    layer.trainable = False

for layer in ChestX_covid.layers:
    layer.trainable = False
    
from tensorflow.keras.layers import Concatenate
concat= Concatenate(axis=1)([net_norm,net_pneumo,net_covid,net_err_norm,net_err_pneumo,net_err_covid])

x = Dense(128, kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(concat)
x = LeakyReLU()(x)
x = Dropout(0.25)(x)

x = Dense(64, kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)

x = Dense(32, kernel_initializer='he_normal', kernel_regularizer=l2(0.1))(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

outputs= Dense(3, activation='softmax')(x)

ChestX_all= Model(inputs= [predicted_input_norm_256,predicted_input_pneumo_256,predicted_input_covid_256,error_input_norm_256,error_input_pneumo_256,error_input_covid_256], outputs= outputs)
ChestX_all.summary()

Model: "model_33"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_164 (InputLayer)         [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_165 (InputLayer)         [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_166 (InputLayer)         [(None, 256, 256, 1  0           []                               
                                )]                                                         

In [146]:
optimizer = Adamax(learning_rate=0.001, beta_1=0.9)
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)



# Compilation du modèle

from gradient_accumulator import GradientAccumulateModel

ChestX_all = GradientAccumulateModel(accum_steps=8, inputs=ChestX_all.input, outputs=ChestX_all.output)

ChestX_all.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
ChestX_all.summary()

Model: "gradient_accumulate_model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_164 (InputLayer)         [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_165 (InputLayer)         [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_166 (InputLayer)         [(None, 256, 256, 1  0           []                               
                                )]                                      

In [149]:
history_ChestX_all = ChestX_all.fit(train_all_generator, epochs=5, validation_data=valid_all_generator)

Epoch 1/5
586/586 [==============================] - 47s 79ms/step - loss: 49831.5820 - accuracy: 0.9680 - val_loss: 49830.1445 - val_accuracy: 1.0000
Epoch 2/5
586/586 [==============================] - 46s 78ms/step - loss: 49828.9922 - accuracy: 0.9603 - val_loss: 49828.1172 - val_accuracy: 1.0000
Epoch 3/5
586/586 [==============================] - 46s 79ms/step - loss: 49827.1367 - accuracy: 0.9821 - val_loss: 49826.5078 - val_accuracy: 1.0000
Epoch 4/5
586/586 [==============================] - 46s 79ms/step - loss: 49826.1055 - accuracy: 0.9795 - val_loss: 49825.3633 - val_accuracy: 1.0000
Epoch 5/5
586/586 [==============================] - 46s 78ms/step - loss: 49824.6289 - accuracy: 0.9880 - val_loss: 49824.4609 - val_accuracy: 1.0000


In [150]:
y_pred_liste=[]
y_true_liste=[]
y_pred_argmax_liste=[]
df_all_test = df_all_test.reset_index(drop=True)

for i in range(len(df_all_test["path"])):
    
    y_pred_norm_256 = df_all_test["outputs_128_to_256_norm"][i]
    y_pred_pneumo_256 = df_all_test["outputs_128_to_256_pneumo"][i]
    y_pred_covid_256 = df_all_test["outputs_128_to_256_covid"][i]

    y_erreur_norm_256 = df_all_test["erreur_128_to_256_norm"][i]
    y_erreur_pneumo_256 = df_all_test["erreur_128_to_256_pneumo"][i]
    y_erreur_covid_256 = df_all_test["erreur_128_to_256_covid"][i]

    pn_256 = np.resize(1000-(y_pred_norm_256),(*(256,256),1))
    pp_256 = np.resize(1000-(y_pred_pneumo_256),(*(256,256),1))
    pc_256 = np.resize(1000-(y_pred_covid_256),(*(256,256),1))

    en_256 = np.resize(1000-(y_erreur_norm_256),(*(256,256),1))
    ep_256 = np.resize(1000-(y_erreur_pneumo_256),(*(256,256),1))
    ec_256 = np.resize(1000-(y_erreur_covid_256),(*(256,256),1))


    pn_256 = (pn_256- np.min(pn_256)) / (np.max(pn_256) - np.min(pn_256))
    pn_256 = np.reshape(pn_256,(*(256,256),1)).astype('float32') 

    pp_256 = (pp_256- np.min(pp_256)) / (np.max(pp_256) - np.min(pp_256))
    pp_256 = np.reshape(pp_256,(*(256,256),1)).astype('float32') 

    pc_256 = (pc_256- np.min(pc_256)) / (np.max(pc_256) - np.min(pc_256))
    pc_256 = np.reshape(pc_256,(*(256,256),1)).astype('float32') 

    en_256 = (en_256- np.min(en_256)) / (np.max(en_256) - np.min(en_256))
    en_256 = np.reshape(en_256,(*(256,256),1)).astype('float32') 

    ep_256 = (ep_256- np.min(ep_256)) / (np.max(ep_256) - np.min(ep_256))
    ep_256 = np.reshape(ep_256,(*(256,256),1)).astype('float32') 

    ec_256 = (ec_256- np.min(ec_256)) / (np.max(ec_256) - np.min(ec_256))
    ec_256 = np.reshape(ec_256,(*(256,256),1)).astype('float32')

    
    

    batch_pn_256 = np.resize(pn_256, (1, *(256,256),1))
    batch_pp_256 = np.resize(pp_256, (1, *(256,256),1))
    batch_pc_256 = np.resize(pc_256, (1, *(256,256),1))
    batch_en_256 = np.resize(en_256, (1, *(256,256),1))
    batch_ep_256 = np.resize(ep_256, (1, *(256,256),1))
    batch_ec_256 = np.resize(ec_256, (1, *(256,256),1))
    
    
    y= df_all_test["label"][i]
    y= np.resize(y, (1, 1))
    batch_y = y

    y_pred = ChestX_all.predict([batch_pn_256,batch_pp_256,batch_pc_256,batch_en_256,batch_ep_256,batch_ec_256],verbose= 0)
    y_pred_argmax = np.argmax(y_pred, axis=1)
    y_pred_liste.extend(y_pred)
    y_pred_argmax_liste.extend(y_pred_argmax)
    y_true_liste.extend(y)
    
y_true_liste = np.reshape(y_true_liste, (732))
y_pred_argmax_liste = np.reshape(y_pred_argmax_liste, (732))
print(pd.crosstab(y_true_liste, y_pred_argmax_liste))

col_0    0    1    2
row_0               
0      249    5    6
1       38  201    8
2        3    8  214


In [ ]:
# We have 91% of accuracy which is not the best possible but is correct. 